In [1]:
AGENT_NAME = r"Victims\3-26-5_PPOc_citylearn_challenge_2022_phase_2_('Building_6',)_gSDE_norm_space_SolarPenaltyReward_deep_net_256_40000.zip"
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = '3-26-5 PPOc Karla results' + '/'
#COL_NAME = '4-2-11'

In [20]:
from stable_baselines3 import SAC, PPO
from citylearn.data import DataSet
from citylearn.citylearn import CityLearnEnv, EvaluationCondition
from citylearn.wrappers import StableBaselines3Wrapper, NormalizedSpaceWrapper
import pandas as pd
import numpy as np
import json

In [3]:
def make_continuous_env(schema, bldg: list = None, single_agent: bool = True, seed:int = 0, T=None, env_kwargs:dict=dict()):
    """Because ART's attacks are designed for supervised learning they one work with ANNs with a single label or head, using multiple buildings adds an action/head for each"""
    
    #TODO support custom rewards
    if bldg is None:
        bldg = list(schema['buildings'].keys())[0] #the first building from the schema's building keys

    kwargs = env_kwargs
    
    env = CityLearnEnv(schema, 
        central_agent=single_agent, 
        buildings=bldg, 
        random_seed=seed,
        episode_time_steps=T,
        **kwargs)
    #Calendar observations are periodically normalized, everything else is min/max normalized 
    env = NormalizedSpaceWrapper(env)
    #provides an interface for SB3
    env = StableBaselines3Wrapper(env)
    return env

In [4]:
def format_kpis(env, eval_condition=None):
    """displays the KPIs from the evnironment's most recent timestep.
    This function can be called after an agent runs in a test env to evaluate performance"""

    if eval_condition is None:
        eval_condition = EvaluationCondition.WITHOUT_STORAGE_BUT_WITH_PARTIAL_LOAD_AND_PV

    kpis = env.evaluate(baseline_condition=eval_condition).pivot(index='cost_function', columns='name', values='value')
    kpis = kpis.dropna(how='all')
    kpis = kpis['District']
    kpis = kpis[kpis != 0]
    return kpis

In [5]:
def eval_agent(env, agent):
    """evaluates the input agent for one episode
    returns a df containing the KPIs, and arrays containing the observations and actions"""
    obs_list = []
    a_list = []

    observations = env.reset()

    while not env.done:
        obs_list.append(observations)
        actions, _ = agent.predict(observations, deterministic=True)
        a_list.append(actions)
        observations, _, _, _ = env.step(actions)
    
    return format_kpis(env), np.array(obs_list), np.array(a_list)

In [6]:
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True,)

schema = DataSet.get_schema(DATASET_NAME)
env = make_continuous_env(schema=schema,  
                                seed=42)

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Linux-5.15.0-78-generic-x86_64-with-glibc2.31 # 85~20.04.1-Ubuntu SMP Mon Jul 17 09:42:39 UTC 2023
- Python: 3.10.13
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: False
- Numpy: 1.25.1
- Gym: 0.21.0



In [7]:
cols = env.observation_names

In [8]:
agent.action_space

Box([0.], [1.], (1,), float32)

In [19]:
agent.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=31, out_features=256, bias=True)
      (1): Tanh()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=31, out_features=256, bias=True)
      (1): Tanh()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=256, out_features=1, bias=True)
  (value_net): Linear(in_features=256, out_features=1, bias=True)
)

In [14]:
hyperparams = {
    'gae_lambda': agent.gae_lambda,
    'batch_size': agent.batch_size,
    #'clip_range': agent.clip_range, #function, not value
    'clip_range_vf': agent.clip_range_vf,
    'ent_coef': agent.ent_coef,
    'gamma': agent.gamma,
    'learning_rate': agent.learning_rate,
    'n_epochs': agent.n_epochs,
    'n_steps': agent.n_steps,
    'target_kl': agent.target_kl,
    'vf_coef': agent.vf_coef,
}

In [15]:
hyperparams

{'gae_lambda': 0.9639143359823201,
 'batch_size': 64,
 'clip_range_vf': None,
 'ent_coef': 1.5635546620789673e-05,
 'gamma': 0.9062274459056439,
 'learning_rate': 8.377741144413337e-05,
 'n_epochs': 8,
 'n_steps': 512,
 'target_kl': None,
 'vf_coef': 0.5}

In [21]:
with open(SAVE_DIR+f'agent hyperparameters.json', 'w') as f:
    json.dump(hyperparams, f)

In [45]:
baseline_kpis, baseline_obs, baseline_a = eval_agent(env,agent)
display(baseline_kpis)

cost_function
annual_peak_average                         1.000000
carbon_emissions_total                      0.858946
cost_total                                  0.813706
daily_one_minus_load_factor_average      7869.833697
daily_peak_average                          0.891421
electricity_consumption_total               0.874358
monthly_one_minus_load_factor_average       0.989392
ramping_average                             0.942670
zero_net_energy                             1.076963
Name: District, dtype: float64

In [47]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename,
                          index_col=0)
    df_kpis['baseline'] = baseline_kpis.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    baseline_kpis.name = 'baseline'
    baseline_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

3-26-5 PPOc Karla results/KPIs.csv created


In [48]:
df_sa = pd.DataFrame(baseline_obs)
df_sa.columns = cols
df_sa['actions'] = baseline_a

In [49]:
df_sa.to_csv(SAVE_DIR+'baseline_obs-a.csv')